# BEAM Stage 3: AA Analysis Demo

## Analyzing AA Trajectories and Comparing with CG

This notebook demonstrates Stage 3:
1. Load AA trajectory from enhanced sampling
2. Transform AA with CG-learned CVs
3. **Visualize CG+AA overlay** (key figure!)
4. Train new TICA on AA data
5. Compare CG vs AA CVs
6. Generate REUS window suggestions

In [ ]:
import sys
sys.path.insert(0, '..')

import numpy as np
import pickle

from beam import (
    train_aa_tica,
    compare_cg_aa_cvs,
    generate_reus_windows,
    plot_cg_aa_overlay,
    plot_tica_projection,
    plot_cv_comparison
)

## Step 1: Load Data

Load CG CV trajectory and AA trajectory.

In [ ]:
# Load CG data (from Stage 2)
with open('../data/cg_tica_model.pkl', 'rb') as f:
    cg_tica_model = pickle.load(f)

cg_features = np.load('../data/demo_cg_traj.npy')
cg_cv = cg_tica_model.transform(cg_features)

print(f"CG CV: {cg_cv.shape}")

# Load AA data
aa_features = np.load('../data/demo_aa_traj.npy')
print(f"AA features: {aa_features.shape}")

## Step 2: Transform AA with CG TICA

See how AA trajectory looks in CG CV space.

In [ ]:
# Transform AA with CG model
aa_in_cg_cv = cg_tica_model.transform(aa_features)

print(f"AA in CG CV space: {aa_in_cg_cv.shape}")

## Step 3: KEY VISUALIZATION - CG+AA Overlay

**This is the most important figure in BEAM!**

It shows:
- Gray background: CG training data
- Colored trajectory: AA enhanced sampling
- Demonstrates that CG CVs successfully guide AA sampling

In [ ]:
plot_cg_aa_overlay(
    cg_cv, 
    aa_in_cg_cv,
    title="AA Enhanced Sampling on CG-learned CV Space",
    save_path='../data/cg_aa_overlay.png'
)

## Step 4: Train TICA on AA Data

Learn AA-specific slow modes.

In [ ]:
# Train AA TICA
aa_tica_model, aa_in_aa_cv = train_aa_tica(
    aa_features,
    lagtime=50,
    dim=2,
    save_path='../data/aa_tica_model.pkl'
)

print(f"\nAA in AA CV space: {aa_in_aa_cv.shape}")

## Step 5: Compare CG vs AA CVs

Compare the slow modes learned from CG vs AA.

In [ ]:
comparison = compare_cg_aa_cvs(cg_tica_model, aa_tica_model)

print("\nComparison status:", comparison['status'])
print("Note:", comparison['note'])

## Step 6: Visualize CV Comparison

In [ ]:
plot_cv_comparison(
    cg_cv,
    aa_in_cg_cv,
    aa_in_aa_cv,
    save_path='../data/cv_comparison.png'
)

## Step 7: Generate REUS Window Suggestions

Based on AA CV range, suggest REUS window placement.

In [ ]:
# Generate windows for CV1
windows_cv1 = generate_reus_windows(
    aa_in_aa_cv,
    n_windows=20,
    buffer=0.1,
    cv_dim=0
)

print("\nWindow centers (first 5):")
print(windows_cv1['window_centers'][:5])

print("\nNote:", windows_cv1['note'])

## Summary

Stage 3 complete!

✅ Loaded AA trajectory from enhanced sampling  
✅ Transformed AA with CG CVs  
✅ **Created key CG+AA overlay visualization**  
✅ Trained TICA on AA data  
✅ Compared CG vs AA CVs (basic)  
✅ Generated REUS window suggestions  

### Key Outputs:

- `cg_aa_overlay.png` - Main figure showing enhanced sampling
- `aa_tica_model.pkl` - AA-trained TICA model
- REUS window suggestions

### Next Steps:

1. Use AA CVs for PMF calculation (via WHAM/MBAR)
2. Set up production REUS simulations
3. Iterate if needed

See `example_pertactin_reus.ipynb` for a real-world case study!